<a href="https://colab.research.google.com/github/SarathSabu/Python-Notebooks/blob/main/Organic_Food_supply_chain_optimization_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Organic Food Supply Chain Optimisation



Install amplpy, pandas and other packages.

In [ ]:
!pip install -q amplpy ampltools

Setup AMPL and select solvers.

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Licensed to Bundle #6741.7193 expiring 20241231: INFO 645 Prescriptive Analytics, Prof. Paul Brooks, Virginia Commonwealth University.


Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read data

In [ ]:
import pandas as pd

# Load data from the Excel file
file_path = '/content/drive/MyDrive/Colab Notebooks/Apple organic food.xlsx'

# Load Transport Costs from Orchard to Preparation Centers
transport_orchard_to_prep = pd.read_excel(file_path, sheet_name='TransportOrchardToPrep', index_col=0)
transport_orchard_to_prep_dict = transport_orchard_to_prep.to_dict()['Transport Cost (Orchard to Prep)']

# Load Preparation Costs
prep_cost = pd.read_excel(file_path, sheet_name='PreparationCosts', index_col=0)
prep_cost_dict = prep_cost.to_dict()['Preparation Cost']

# Load Monthly Capacity
capacity = pd.read_excel(file_path, sheet_name='MonthlyCapacity', index_col=0)
capacity_dict = capacity.to_dict()['Capacity (Pounds)']

# Load Demand at Stores
demand = pd.read_excel(file_path, sheet_name='StoreDemand', index_col=0)
demand_dict = demand.to_dict()['Demand (Pounds)']

# Load Transport Costs from Preparation Centers to Stores
transport_cost = pd.read_excel(file_path, sheet_name='TransportPrepToStores', index_col=0)
transport_cost_dict = transport_cost.to_dict()

# Display the data to verify correctness
print("Transport Orchard to Preparation Centers:", transport_orchard_to_prep_dict)
print("Preparation Costs:", prep_cost_dict)
print("Capacity:", capacity_dict)
print("Demand:", demand_dict)
print("Transport Costs Prep Centers to Stores:", transport_cost_dict)

Transport Orchard to Preparation Centers: {1: 0.45, 2: 1.0, 3: 1.62}
Preparation Costs: {1: 0.15, 2: 0.2, 3: 0.18}
Capacity: {1: 300, 2: 500, 3: 800}
Demand: {1: 300, 2: 500, 3: 400, 4: 200}
Transport Costs Prep Centers to Stores: {'Organic Orchard': {1: 0.8, 2: 1.2, 3: 0.2}, 'Fresh & Local': {1: 1.1, 2: 1.1, 3: 1.4}, 'Healthy Pantry': {1: 0.7, 2: 0.5, 3: 1.3}, "Season's Harvest": {1: 1.4, 2: 1.4, 3: 1.7}}



Define model.

In [ ]:
ampl.eval ('''

reset;
set P;  # Set of preparation centers
set S;  # Set of specialty stores

param transport_cost_orchard_to_prep {i in P};  # Transportation cost from orchard to preparation center i
param prep_cost {i in P};                       # Preparation cost at preparation center i
param capacity {i in P};                        # Monthly capacity of preparation center i
param transport_cost {i in P, j in S};          # Transportation cost from preparation center i to store j
param demand {j in S};                          # Monthly demand at store j

var x {i in P, j in S} >= 0;  # Pounds of apples transported from preparation center i to store j
var y {i in P} >= 0;          # Pounds of apples transported from orchard to preparation center i


# Objective: Minimize total cost (transportation + preparation + shipping)
minimize Total_Cost:sum {i in P} (transport_cost_orchard_to_prep[i] * y[i] + prep_cost[i] * y[i] + sum {j in S} transport_cost[i,j] * x[i,j]);

subject to
Demand_Constraint {j in S}:sum {i in P} x[i,j] >= demand[j];
Capacity_Constraint {i in P}: sum {j in S} x[i,j] <= capacity[i];
Orchard_Flow_Constraint {i in P}:sum {j in S} x[i,j] = y[i];

''')

Provide data to the model.

In [ ]:
store_mapping = {
    'Organic Orchard': 1,
    'Fresh & Local': 2,
    'Healthy Pantry': 3,
    "Season's Harvest": 4
}

# Create a new dictionary with integer keys for the stores
transport_cost_mapped = {}

for store_name, i_data in transport_cost_dict.items():
    store_index = store_mapping[store_name]  # Get the store index from the mapping
    for prep_center, cost in i_data.items():
        if prep_center not in transport_cost_mapped:
            transport_cost_mapped[prep_center] = {}
        transport_cost_mapped[prep_center][store_index] = cost

# Display the remapped transport cost dictionary to verify
print(transport_cost_mapped)


{1: {1: 0.8, 2: 1.1, 3: 0.7, 4: 1.4}, 2: {1: 1.2, 2: 1.1, 3: 0.5, 4: 1.4}, 3: {1: 0.2, 2: 1.4, 3: 1.3, 4: 1.7}}


In [ ]:
ampl.set['P'] = [1, 2, 3]
ampl.set['S'] = [1, 2, 3, 4]

# Assign data from Python to AMPL parameters
ampl.param['transport_cost_orchard_to_prep'] = transport_orchard_to_prep_dict
ampl.param['prep_cost'] = prep_cost_dict
ampl.param['capacity'] = capacity_dict
ampl.param['demand'] = demand_dict

# Assign the remapped transport cost data to AMPL parameters
for i in range(1, 4):  # Loop over preparation centers
    for j in range(1, 5):  # Loop over stores (with integer indices)
        ampl.param['transport_cost'][i, j] = transport_cost_mapped[i][j]





Display problem formulation.

In [ ]:
ampl.eval('''expand;''')
# ampl.eval('''expand x;''')
# ampl.eval('''expand y;''')


minimize Total_Cost:
	0.8*x[1,1] + 1.1*x[1,2] + 0.7*x[1,3] + 1.4*x[1,4] + 1.2*x[2,1] + 
	1.1*x[2,2] + 0.5*x[2,3] + 1.4*x[2,4] + 0.2*x[3,1] + 1.4*x[3,2] + 
	1.3*x[3,3] + 1.7*x[3,4] + 0.6*y[1] + 1.2*y[2] + 1.8*y[3];

subject to Demand_Constraint[1]:
	x[1,1] + x[2,1] + x[3,1] >= 300;

subject to Demand_Constraint[2]:
	x[1,2] + x[2,2] + x[3,2] >= 500;

subject to Demand_Constraint[3]:
	x[1,3] + x[2,3] + x[3,3] >= 400;

subject to Demand_Constraint[4]:
	x[1,4] + x[2,4] + x[3,4] >= 200;

subject to Capacity_Constraint[1]:
	x[1,1] + x[1,2] + x[1,3] + x[1,4] <= 300;

subject to Capacity_Constraint[2]:
	x[2,1] + x[2,2] + x[2,3] + x[2,4] <= 500;

subject to Capacity_Constraint[3]:
	x[3,1] + x[3,2] + x[3,3] + x[3,4] <= 800;

subject to Orchard_Flow_Constraint[1]:
	x[1,1] + x[1,2] + x[1,3] + x[1,4] - y[1] = 0;

subject to Orchard_Flow_Constraint[2]:
	x[2,1] + x[2,2] + x[2,3] + x[2,4] - y[2] = 0;

subject to Orchard_Flow_Constraint[3]:
	x[3,1] + x[3,2] + x[3,3] + x[3,4] - y[3] = 0;



Set solver and solve.

In [ ]:
ampl.setOption('solver', 'cplex')
ampl.solve()

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3040
8 simplex iterations


Print solution and results.

In [ ]:
# Get and display results
x = ampl.var['x'].get_values()
y = ampl.var['y'].get_values()
total_cost = ampl.obj['Total_Cost'].value()
print("Total Cost:", total_cost)
print("x (apples shipped):", x)
print("y (apples transported from orchard):", y)

Total Cost: 3040.0
x (apples shipped):    index0       index1    |    x.val    
     1              1       |      0      
     1              2       |     100     
     1              3       |      0      
     1              4       |     200     
     2              1       |      0      
     2              2       |     100     
     2              3       |     400     
     2              4       |      0      
     3              1       |     300     
     3              2       |     300     
     3              3       |      0      
     3              4       |      0      

y (apples transported from orchard):    index0    |    y.val    
     1       |     300     
     2       |     500     
     3       |     600     



The optimal solution is to produce and ship 300 apples to preparation centre 1, 500 apples to preparation centre 2 and 600 apples to preparation centre 3 from the orchard. From preparation centre 1, 100 apples should be sent to Fresh & Local and 200 apples to Season's harvest speciality stores. From preparation centre 2, 100 apples should be sent to Fresh & Local and 400 apples to Healthy pantry speciality stores. From preparation centre 3, 300 apples should be sent to Organic Orchard and 300 apples to the Fresh & Local speciality stores. The total cost will be $3040.